_We're going to be using pandas an json, so lets start with that._

In [1]:
import json
import pandas as pd
import os
from pandas.io.json import json_normalize

# Looking at the json files

_Before we do any real work, can we list the files from our target directory?_

In [2]:
!pwd
! ls ../APIs

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json


_Nice! Let's examine a json file_

In [3]:
!head ../APIs/yahoo_weather.json

{
    "query": {
        "count": 1,
        "created": "2017-07-29T23:51:25Z",
        "lang": "en-US",
        "diagnostics": {
            "publiclyCallable": "true",
            "url": {
                "execution-start-time": "1",
                "execution-stop-time": "5",


# Exploring JSON from the Yahoo Weather API

_Read a json file from disk (we'll use ../API/<tbd>.json
We should reall use the yahoo api output, since we know how to convert that to a csv_

Source [Andy Hayden on ghthub.io](http://hayd.github.io/2013/pandas-json)

In [4]:
df = pd.read_json('../APIs/yahoo_weather.json')
df

,query
count,1
created,2017-07-29T23:51:25Z
diagnostics,"{'publiclyCallable': 'true', 'url': {'executio..."
lang,en-US
results,"{'channel': {'units': {'distance': 'mi', 'pres..."


_Not much of a dataframe. Let's re-read and nomralize, drilling into the results section._

Read file with json Library. Source: [Scott Robinson's Stack Abuse Article ](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)

In [35]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']) 
yahoo_weather_df

,count,created,diagnostics.build-version,diagnostics.publiclyCallable,diagnostics.service-time,diagnostics.url.content,diagnostics.url.execution-start-time,diagnostics.url.execution-stop-time,diagnostics.url.execution-time,diagnostics.user-time,...,results.channel.location.region,results.channel.title,results.channel.ttl,results.channel.units.distance,results.channel.units.pressure,results.channel.units.speed,results.channel.units.temperature,results.channel.wind.chill,results.channel.wind.direction,results.channel.wind.speed
0,1,2017-07-29T23:51:25Z,2.0.164,true,4,http://weather-ydn-yql.media.yahoo.com:4080/v3...,1,5,4,6,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_One row, 50 columns. Let's look at data types._

In [37]:
yahoo_weather_df.dtypes

count                                     int64
created                                  object
diagnostics.build-version                object
diagnostics.publiclyCallable             object
diagnostics.service-time                 object
diagnostics.url.content                  object
diagnostics.url.execution-start-time     object
diagnostics.url.execution-stop-time      object
diagnostics.url.execution-time           object
diagnostics.user-time                    object
lang                                     object
results.channel.astronomy.sunrise        object
results.channel.astronomy.sunset         object
results.channel.atmosphere.humidity      object
results.channel.atmosphere.pressure      object
results.channel.atmosphere.rising        object
results.channel.atmosphere.visibility    object
results.channel.description              object
results.channel.image.height             object
results.channel.image.link               object
results.channel.image.title             

_That's a lot of data, all of type object. For the Yahoo Weather API, most of the good stuff lives in_ results.channel
_Let's dig deeper_

In [38]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_df = json_normalize(raw_json['query']['results']['channel']) 
yahoo_weather_df

,astronomy.sunrise,astronomy.sunset,atmosphere.humidity,atmosphere.pressure,atmosphere.rising,atmosphere.visibility,description,image.height,image.link,image.title,...,location.region,title,ttl,units.distance,units.pressure,units.speed,units.temperature,wind.chill,wind.direction,wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


_Looking more like a dataframe.  Let's look at fields & datatypes_

In [39]:
yahoo_weather_df.dtypes

astronomy.sunrise        object
astronomy.sunset         object
atmosphere.humidity      object
atmosphere.pressure      object
atmosphere.rising        object
atmosphere.visibility    object
description              object
image.height             object
image.link               object
image.title              object
image.url                object
image.width              object
item.condition.code      object
item.condition.date      object
item.condition.temp      object
item.condition.text      object
item.description         object
item.forecast            object
item.guid.isPermaLink    object
item.lat                 object
item.link                object
item.long                object
item.pubDate             object
item.title               object
language                 object
lastBuildDate            object
link                     object
location.city            object
location.country         object
location.region          object
title                    object
ttl     

_Everything's an object.  If we want to do any real analysis, we're going to need to do some type conversion. Also note, that the field names have a hierarchy to them.  So maybe we can drill deeper into just the fields we want_

_Drill Down: atmosphere_

In [31]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['atmosphere']) 
yahoo_weather_atmosphere_df

,humidity,pressure,rising,visibility
0,58,1006.0,0,16.1


_Drill Down: image_

In [41]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_image_df = json_normalize(raw_json['query']['results']['channel']['image']) 
yahoo_weather_image_df

,height,link,title,url,width
0,18,http://weather.yahoo.com,Yahoo! Weather,http://l.yimg.com/a/i/brand/purplelogo//uh/us/...,142


_Drill Down: item_

In [45]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_item_df = json_normalize(raw_json['query']['results']['channel']['item']) 
yahoo_weather_item_df

,condition.code,condition.date,condition.temp,condition.text,description,forecast,guid.isPermaLink,lat,link,long,pubDate,title
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny,"<![CDATA[<img src=""http://l.yimg.com/a/i/us/we...","[{'code': '34', 'date': '29 Jul 2017', 'day': ...",false,37.371609,http://us.rd.yahoo.com/dailynews/rss/weather/C...,-122.038254,"Sat, 29 Jul 2017 04:00 PM PDT","Conditions for Sunnyvale, CA, US at 04:00 PM PDT"


_Drill Down: item.forecast_

In [46]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_forecast_df = json_normalize(raw_json['query']['results']['channel']['item']['forecast']) 
yahoo_weather_forecast_df

,code,date,day,high,low,text
0,34,29 Jul 2017,Sat,78,58,Mostly Sunny
1,32,30 Jul 2017,Sun,84,60,Sunny
2,32,31 Jul 2017,Mon,87,61,Sunny
3,32,01 Aug 2017,Tue,88,64,Sunny
4,34,02 Aug 2017,Wed,90,66,Mostly Sunny
5,30,03 Aug 2017,Thu,88,68,Partly Cloudy
6,34,04 Aug 2017,Fri,88,67,Mostly Sunny
7,32,05 Aug 2017,Sat,86,66,Sunny
8,34,06 Aug 2017,Sun,83,63,Mostly Sunny
9,34,07 Aug 2017,Mon,79,61,Mostly Sunny


_Drill Down:condition_

In [47]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_conditions_df = json_normalize(raw_json['query']['results']['channel']['item']['condition']) 
yahoo_weather_conditions_df

,code,date,temp,text
0,32,"Sat, 29 Jul 2017 04:00 PM PDT",75,Sunny


_Drill Down: wind_

In [48]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

yahoo_weather_atmosphere_df = json_normalize(raw_json['query']['results']['channel']['wind']) 
yahoo_weather_atmosphere_df

,chill,direction,speed
0,75,355,18


# Exploring JSON from the Open Weather Map API

In [55]:
!pwd
! ls ../APIs
! cat ../APIs/openweathermap.json

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
2yahoo.json	     wsdot.TrafficFlows.json  yahoo_weather1line.json
API_index.csv	     wsdot.Traveltimes.json   yahoo_weather2.json
openweathermap.json  yahoo_weather	      yahoo_weather.json
{
    "coord": {
        "lon": 139.01,
        "lat": 35.02
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",
            "icon": "01n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 285.514,
        "pressure": 1013.75,
        "humidity": 100,
        "temp_min": 285.514,
        "temp_max": 285.514,
        "sea_level": 1023.22,
        "grnd_level": 1013.75
    },
    "wind": {
        "speed": 5.52,
        "deg": 311
    },
    "clouds": {
        "all": 0
    },
    "dt": 1485792967,
    "sys": {
        "message": 0.0025,
        "country": "JP",
        "sunrise": 1485726240,
        "sunset"

In [51]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_df = json_normalize(raw_json) 
openweathermap_df

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,0,200,35.02,139.01,1485792967,1907296,1013.75,100,1013.75,...,285.514,285.514,Tawarano,JP,0.0025,1485726240,1485763863,"[{'id': 800, 'main': 'Clear', 'description': '...",311,5.52


In [53]:
openweathermap_df.dtypes

base                object
clouds.all           int64
cod                  int64
coord.lat          float64
coord.lon          float64
dt                   int64
id                   int64
main.grnd_level    float64
main.humidity        int64
main.pressure      float64
main.sea_level     float64
main.temp          float64
main.temp_max      float64
main.temp_min      float64
name                object
sys.country         object
sys.message        float64
sys.sunrise          int64
sys.sunset           int64
weather             object
wind.deg             int64
wind.speed         float64
dtype: object

_May of the data types are already numeric.  Pretty cool. Let's dive into the weather object._

In [56]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

openweathermap_weather_df = json_normalize(raw_json['weather']) 
openweathermap_weather_df

,description,icon,id,main
0,clear sky,01n,800,Clear


# ---------------------------------------< NEW STUFF >---------------------------------

# ---------------------------------------< OLD STUFF >---------------------------------

# Multi-Line Considerations
Okay, we want dataframes with multiple rows in them, but each json object will be only one row.
We have two options ....
1. Have a ton of small single-object files, all living in a folder.  Check file count, read newest file whenever count goes up
2. Have one big json file, one joson object per line, that we append to.

I think what we _really_ want, is a kafka consumer that was pandas-aware. So when it gets a new message, it makes a json object and appends it to an existing dataframe.
- ehh, that kinda breaks our architecture. We don't want to "require" people to use data frames (do we)?.  It's more ELT-ish to save the transform to the very end.  Delivery the json's, full-stop; then load them into pandas (or into SQL or whatever .... as you choose)

So .....
- Let's stick with the assumption that there will be json-on-disk. Probably separate files. 

### Assuming we have on json file per object ...
- Iterate over alljson files in the folder
    - read the file
    - make a one-record data frame, normalized the way youwant.
    - append to a larger data frame.

In [ ]:
top_folder = "../APIs/yahoo_weather"
first_time = True

for filename in os.listdir(top_folder):
    filepath = top_folder + "/" + filename


    with open(filepath) as json_file:
      raw_json = json.load(json_file)

    if (first_time):
 
        first_time = False
        bigdf = json_normalize(raw_json) 
    else:

        new_json = json_normalize(raw_json)
        bigdf = bigdf.append(new_json, ignore_index=True)
    
bigdf

In [ ]:
bigdf.dtypes

### Multi-line examples
Here's a way to read multile "files-worth" of json objects into a dataframe
- First, remove all the new-lines, and/or carriage returns, so that you json object is all on onel ine of text.
- Next, concatenate your "one-line json's" into a single file.
- Use the Pandas read_json() function with the lines flag set to true.

In [ ]:
# df3 = pd.read_json('../APIs/2yahoo.json')
df3 = pd.read_json('../APIs/2yahoo.json', lines=True)
df3

In [ ]:
df = pd.read_json('../APIs/yahoo_weather.json')
df
raw_json = df.to_json
raw_json

In [ ]:



normal_json = json_normalize(raw_json) 
normal_json

In [ ]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

raw_json

normal_json = json_normalize(raw_json['query']) 
normal_json

## Future Work
- Use Postman to generate more yahoo weather json files, then cocatenate them into one dataframe
- Try other json files
- Get to the point where you have multiple rows to play with. 